# JSONデータをPandasDF化する

## ①シンプルな構造のJSON

In [1]:
import pandas as pd
#!wget 'https://raw.githubusercontent.com/yoichiro0903n/blue/main/cars.csv'
jsonfile = '01simple.json'
df = pd.read_json(jsonfile)
df


,id,UP_TIME,POWER,TEMP,ERR_CD
0,1000,0,948,250,NaN
1,1000,1,945,251,1.0


pandasでJSON文字列・ファイルを読み込み（read_json） | note.nkmk.me https://note.nkmk.me/python-pandas-read-json/

## ②ネスト構造のJSON

pd.read_jsonだとうまく読めない

In [2]:
jsonfile='02notnormal.json'
df=pd.read_json(jsonfile)
df

,id,sensor,code
0,"{'M_CD': 1000, 'UP_TIME': 0}","{'POWER': 948, 'TEMP': 250}",NaN
1,"{'M_CD': 1000, 'UP_TIME': 1}","{'POWER': 945, 'TEMP': 251}",{'ERR_CD': 1}


pd.json_normalizeを使う

In [3]:
import json
jsonfile='02notnormal.json'
with open(jsonfile, encoding='utf-8') as f:
    d = json.load(f)
#print(d)
df=pd.json_normalize(d)
df

,id.M_CD,id.UP_TIME,sensor.POWER,sensor.TEMP,code.ERR_CD
0,1000,0,948,250,NaN
1,1000,1,945,251,1.0


PandasでJSON形式の列データを複数列に展開する - そうなんでげす https://www.soudegesu.com/post/python/flatten-json-column-with-pandas/

### ③ネスト構造とリストを含むJSON

pd.read_jsonだとネスト構造が展開されない

In [4]:
jsonfile='03list.json'
df=pd.read_json(jsonfile)
df

,id,sensor,code
0,"{'M_CD': 1000, 'UP_TIME': 0}","{'POWER': 948, 'TEMP': 250}",NaN
1,"{'M_CD': 1000, 'UP_TIME': 1}","{'POWER': 945, 'TEMP': 251}","[{'ERR_CD': 1, 'MESSAGE': 'part1'}]"
2,"{'M_CD': 1000, 'UP_TIME': 2}","{'POWER': 943, 'TEMP': 255}","[{'ERR_CD': 2, 'MESSAGE': 'part2'}, {'ERR_CD':..."


pd.json_normalizeではリスト構造が残ってしまう

In [5]:
import json
jsonfile='03list.json'
with open(jsonfile, encoding='utf-8') as f:
    d = json.load(f)
#print(d)
df=pd.json_normalize(d)
df

,id.M_CD,id.UP_TIME,sensor.POWER,sensor.TEMP,code
0,1000,0,948,250,NaN
1,1000,1,945,251,"[{'ERR_CD': 1, 'MESSAGE': 'part1'}]"
2,1000,2,943,255,"[{'ERR_CD': 2, 'MESSAGE': 'part2'}, {'ERR_CD':..."


リストをフラット化するサンプルプログラム

In [6]:
# 入力JSONファイル名
import pandas as pd
import json
jsonfile = '03list.json'
# 列をフラット化する際の区切り文字
sep = '.'


# フラット化
def flatten(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        # 列名の生成
        new_key = parent_key + sep + k if parent_key else k
        # 辞書型項目のフラット化
        if isinstance(v, dict):
            items.extend(flatten(v, new_key, sep=sep).items())
        # リスト項目のフラット化
        elif isinstance(v, list):
            new_key_tmp = new_key
            for i, elm in enumerate(v):
                new_key = new_key_tmp + sep + str(i)
                # リストの中の辞書
                if isinstance(elm, dict):
                    items.extend(flatten(elm, new_key, sep=sep).items())
                # 単なるリスト
                else:
                    items.append((new_key, elm))
        # 値追加
        else:
            items.append((new_key, v))
    return dict(items)


# JSONファイルを読込
with open(jsonfile, encoding='utf-8') as f:
    d = json.load(f)


# フラット化
dlist = []
for di in d:
    dlist.append(flatten(di, sep=sep))

# print(dlist)
# フラット化された辞書をpandasデータフレームに変換
df = pd.DataFrame.from_dict(dlist)
df


,id.M_CD,id.UP_TIME,sensor.POWER,sensor.TEMP,code.0.ERR_CD,code.0.MESSAGE,code.1.ERR_CD,code.1.MESSAGE
0,1000,0,948,250,NaN,NaN,NaN,NaN
1,1000,1,945,251,1.0,part1,NaN,NaN
2,1000,2,943,255,2.0,part2,3.0,part3


python - How to flatten multilevel/nested JSON? - Stack Overflow https://stackoverflow.com/questions/51359783/how-to-flatten-multilevel-nested-json

## ④ルートが読み出しルートではないJSON

pd.read_jsonだとネスト構造が展開されない

In [7]:
jsonfile='04notroot.json'
df=pd.read_json(jsonfile)
df

,total_rows,rows
0,3,"{'id': {'M_CD': 1000, 'UP_TIME': 0}, 'sensor':..."
1,3,"{'id': {'M_CD': 1000, 'UP_TIME': 1}, 'sensor':..."
2,3,"{'id': {'M_CD': 1000, 'UP_TIME': 2}, 'sensor':..."


pd.json_normalizeだとリストが展開されない。

In [8]:
import json
jsonfile='04notroot.json'
with open(jsonfile, encoding='utf-8') as f:
    d = json.load(f)
#print(d)
df=pd.json_normalize(d)
df

,total_rows,rows
0,3,"[{'id': {'M_CD': 1000, 'UP_TIME': 0}, 'sensor'..."


flattenだと3行に展開されない

In [9]:

#フラット化された辞書をpandasデータフレームに変換
df = pd.DataFrame.from_dict([flatten(d, sep=sep)])
df

,total_rows,rows.0.id.M_CD,rows.0.id.UP_TIME,rows.0.sensor.POWER,rows.0.sensor.TEMP,rows.1.id.M_CD,rows.1.id.UP_TIME,rows.1.sensor.POWER,rows.1.sensor.TEMP,rows.1.code.0.ERR_CD,rows.1.code.0.MESSAGE,rows.2.id.M_CD,rows.2.id.UP_TIME,rows.2.sensor.POWER,rows.2.sensor.TEMP,rows.2.code.0.ERR_CD,rows.2.code.0.MESSAGE,rows.2.code.1.ERR_CD,rows.2.code.1.MESSAGE,rows.2.isActive
0,3,1000,0,948,250,1000,1,945,251,1,part1,1000,2,943,255,2,part2,3,part3,False


読み出しルートを変更するサンプル

In [10]:
jsonfile = '04notroot.json'
rowsroot = "rows"
# JSONファイルを読込
with open(jsonfile, encoding='utf-8') as f:
    d = json.load(f)

# df化したい辞書リストのルート項目を指定
if rowsroot != '':
    d = d[rowsroot]

# フラット化
dlist = []
for di in d:
    dlist.append(flatten(di, sep=sep))

# フラット化された辞書をpandasデータフレームに変換
df = pd.DataFrame.from_dict(dlist)
df


,id.M_CD,id.UP_TIME,sensor.POWER,sensor.TEMP,code.0.ERR_CD,code.0.MESSAGE,code.1.ERR_CD,code.1.MESSAGE,isActive
0,1000,0,948,250,NaN,NaN,NaN,NaN,NaN
1,1000,1,945,251,1.0,part1,NaN,NaN,NaN
2,1000,2,943,255,2.0,part2,3.0,part3,False


pandasでJSON文字列・ファイルを読み込み（read_json） | note.nkmk.me https://note.nkmk.me/python-pandas-read-json/

# JSONファイルのpandasDF化関数（完成版）

In [11]:

import pandas as pd
import json

# フラット化


def flatten(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        # 列名の生成
        new_key = parent_key + sep + k if parent_key else k
        # 辞書型項目のフラット化
        if isinstance(v, dict):
            items.extend(flatten(v, new_key, sep=sep).items())
        # リスト項目のフラット化
        elif isinstance(v, list):
            new_key_tmp = new_key
            for i, elm in enumerate(v):
                new_key = new_key_tmp + sep + str(i)
                # リストの中の辞書
                if isinstance(elm, dict):
                    items.extend(flatten(elm, new_key, sep=sep).items())
                # 単なるリスト
                else:
                    items.append((new_key, elm))
        # 値追加
        else:
            items.append((new_key, v))
    return dict(items)


def flattenJsonFile(jsonfile, rowsroot, sep='.'):
    """
    JSONファイルを読み込み2次元のpandas DataFrameに変換する

    Parameters
    ----------
    jsonfile : string
        JSONファイルパス
    rowsroot : string
        フラット化するルートエレメント名。トップからでいい場合は空文字を入力する
    sep : string
        ノーマライズされていないエレメントを区切る文字

    Returns
    -------
    df : pandas.DataFrame
        フラット化されたpandas DataFrame
    """
   # JSONファイルを読込
    with open(jsonfile, encoding='utf-8') as f:
        d = json.load(f)

    # df化したい辞書リストのルート項目を指定
    if rowsroot != '':
        d = d[rowsroot]

    # フラット化
    dlist = []
    for di in d:
        dlist.append(flatten(di, sep=sep))

    # フラット化された辞書をpandasデータフレームに変換
    return pd.DataFrame.from_dict(dlist)



①から④までのすべてのファイルを処理可能

In [12]:
flattenJsonFile('01simple.json','','.')

,id,UP_TIME,POWER,TEMP,ERR_CD
0,1000,0,948,250,NaN
1,1000,1,945,251,1.0


In [13]:
flattenJsonFile('02notnormal.json','','.')

,id.M_CD,id.UP_TIME,sensor.POWER,sensor.TEMP,code.ERR_CD
0,1000,0,948,250,NaN
1,1000,1,945,251,1.0


In [14]:
flattenJsonFile('03list.json','','.')

,id.M_CD,id.UP_TIME,sensor.POWER,sensor.TEMP,code.0.ERR_CD,code.0.MESSAGE,code.1.ERR_CD,code.1.MESSAGE
0,1000,0,948,250,NaN,NaN,NaN,NaN
1,1000,1,945,251,1.0,part1,NaN,NaN
2,1000,2,943,255,2.0,part2,3.0,part3


In [15]:


flattenJsonFile('04notroot.json','rows','.')

,id.M_CD,id.UP_TIME,sensor.POWER,sensor.TEMP,code.0.ERR_CD,code.0.MESSAGE,code.1.ERR_CD,code.1.MESSAGE,isActive
0,1000,0,948,250,NaN,NaN,NaN,NaN,NaN
1,1000,1,945,251,1.0,part1,NaN,NaN,NaN
2,1000,2,943,255,2.0,part2,3.0,part3,False


In [16]:
import sys

print(sys.version)
print(pd.__version__)
import json
print(json.__version__)


3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]
1.5.2
2.0.9
